# Retrieval Experiments

Relevance of context returned from search greatly impacts the quality of our RAG system. This notebook will explore what retrieval methods are available to us with the goal of producing the most relevant content to a users query.

In [1]:
import sys 
import subprocess

# get root of current repo and add to our path
root_dir = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL).decode("utf-8").strip()

sys.path.append(root_dir)

## Setup 

Before we get started, let's initialize our clients, models, and test params for re-use throguhout this notebook

In [5]:
# for consistency, define a set of test queries 
test_queries = [
    "tell me about yourself", 
    "what is your educational background", 
    "why are you seeking a new position", 
    "what experience do you have with data pipelines", 
]

In [3]:
from sentence_transformers import SentenceTransformer

# instantiate the model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/Users/srmarshall/.virtualenvs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# from utils.postgres import PostgresClient
import os 
import psycopg2
from pgvector.psycopg2 import register_vector


class PostgresClient:
    def __init__(self, pg_host: str, pg_user: str, pg_password: str, pg_db: str):
        self.pg_host = pg_host
        self.pg_user = pg_user
        self.pg_password = pg_password
        self.pg_db = pg_db

    def _make_conn(self):
        try:
            conn = psycopg2.connect(
                host=self.pg_host,
                user=self.pg_user,
                password=self.pg_password,
                dbname=self.pg_db,
            )
            register_vector(conn)
            return conn
        except Exception as e:
            print(f"Error connecting to Postgres: {str(e)}")
            return None

    def insert_content_embeddings(self, data: list):
        try:
            conn = self._make_conn()
            cursor = conn.cursor()
            for record in data:
                try:
                    cursor.execute(
                        """
                                INSERT INTO content_embeddings(document_id, tags, clean_text, embedding)
                                    VALUES(%s, %s, %s, %s);
                            """,
                        (
                            record["document_id"],
                            record["tags"],
                            record["clean_text"],
                            record["embedding"],
                        ),
                    )
                except Exception as e:
                    print(f"Error inserting record: {str(e)}")
            conn.commit()
            cursor.close()
            conn.close()
        except Exception as e:
            print(f"Error connecting to Postgres: {str(e)}")

    def semantic_search(self, query_embedding: list, n_results: int = 5):
        # convert query embedidng to the proper format
        query_embedding_str = ", ".join(map(str, query_embedding))

        # build query
        search = f"SELECT document_id, tags, clean_text, 1 - (embedding <=> '[{query_embedding_str}]') as similarity_score FROM content_embeddings ORDER BY embedding <=> '[{query_embedding_str}]' LIMIT {n_results};"

        # establish a connection to the DB
        conn = self._make_conn()
        try:
            cursor = conn.cursor()
            cursor.execute(search)
            results = cursor.fetchall()
            return results
        except Exception as e:
            print(f"Error conduting semantic search: {str(e)}")
            return None
        finally:
            cursor.close()

    def hybrid_search(self, query_text: str, query_embedding: list, n_results: int = 5, similarity_threshold: float = 0.3):
    keywords = [word.lower() for word in query_text.split()]
    query_embedding_str = ", ".join(map(str, query_embedding))

    conn = self._make_conn()
    if not conn:
        return None

    try:
        cursor = conn.cursor()

        for keyword in keywords:
            cursor.execute(f"""
                SELECT * FROM (
                    SELECT 
                        document_id,
                        tags,
                        clean_text,
                        1 - (embedding <=> '[{query_embedding_str}]') AS similarity_score,
                        (
                            SELECT MAX(similarity(tag, %s::text))
                            FROM unnest(tags) AS tag
                            WHERE similarity(tag, %s::text) > %s
                        ) AS tag_score
                    FROM content_embeddings
                ) AS ranked_results
                ORDER BY similarity_score + COALESCE(tag_score, 0) DESC
                LIMIT %s;
            """, (keyword, keyword, similarity_threshold, n_results))

            rows = cursor.fetchall()
            if rows:
                return rows

        return []
    except Exception as e:
        print(f"Error conducting hybrid search: {str(e)}")
        return None
    finally:
        cursor.close()
        conn.close()




# initialize a postgres client 
pg = PostgresClient(
    pg_host=os.getenv("PG_HOST"),
    pg_user=os.getenv("PG_USER"),
    pg_password=os.getenv("PG_PASSWORD"),
    pg_db="resume_rag"
)

## Cosine Similarity Search 

Let's start simple with `pgvector` cosine similarity search. This approach will take a query embedding and calculate the cosine similarity between each observation in our database. 


**Strengths:**
- Excellent response to why searching and tell me about yourself. There are documents in the knowledge base speaking directly to these topics and this metric does a good job surfacing them 

**Weaknesses:**
- Results for educational background are not what we want. As expected, there is some confusion between "Education Analytics" and formal education at university
- Results for experience with pipelines misses the mark a bit. The first result is great, but the subsequent two aren't really related to the question. 

**Next Steps:**
- Find a way to differentiate Education Analytics from "education" when referring to college or unitversity
- Add additional documents that speak to specific experience with technologies or projects. 


In [17]:
for query in test_queries:
    print("Test Query: ", query)

    query_embedding = model.encode(query)
    results = pg.semantic_search(query_embedding, n_results=3)
    print("Top 3 Results:")
    for result in results:
        print(f"- {result[2]}")
    print("\n")


Test Query:  tell me about yourself
Top 3 Results:
- skills that directly support her current work in data engineering and machine learning while studying psychology sophie developed a deep interest in the mechanisms of human cognition which naturally led her to explore fields like artificial intelligence and machine learning her coursework in
- sophie marshall graduated from the university of wisconsin madison in 2022 with a bachelor of science in psychology and economics with a mathematical emphasis her academic training reflects a strong interdisciplinary foundation in human cognition data modeling and statistical analysis skills that
- her coursework in economics paired with a focus on mathematical modeling helped her build a solid foundation in systems thinking multivariate analysis and data driven decision making during her undergraduate years sophie was a division i athlete competing as a four year member of the wisconsin


Test Query:  what is your educational background
Top 3 

## Tag Augementation 

We want some way to bolster responses if a word from the query either exactly matches or closely matches a word from a tag. The hope is that this addresses the confusion between "Education Analytics" and education as it relates to college. 

An additional layer of security we can add later down the line is to explicitly provide this information in the system prompt. However, this doesnt address the problem of irrelevant information taking the place of relevant information in the 3 returned DB results.

In [30]:
education_query = test_queries[1]

# cosine search with a bigger set of results 
hybrid_search_results = pg.hybrid_search(education_query, query_embedding, n_results=30)

Error conducting hybrid search: column "similarity_score" does not exist
LINE 13:                     ORDER BY similarity_score + COALESCE(tag...
                                      ^



In [27]:
education_query.split(" ")

['what', 'is', 'your', 'educational', 'background']

In [25]:
for result in cosine_similarity_results:
    print(result[1])

['job', 'professional', 'experience', 'work history']
['summary', 'professional summary', 'elevator pitch']
['job', 'professional', 'experience', 'work history']
['job', 'professional', 'experience', 'work history']
['summary', 'professional summary', 'elevator pitch']
['job', 'professional', 'experience', 'work history']
['education', 'university', 'college', 'degree']
['job', 'professional', 'experience', 'work history']
['job', 'professional', 'experience', 'work history']
['job', 'professional', 'experience', 'work history']
['job', 'professional', 'experience', 'work history']
['summary', 'professional summary', 'elevator pitch']
['job', 'professional', 'experience', 'work history']
['job', 'professional', 'experience', 'work history', 'internship']
['job', 'professional', 'experience', 'work history', 'internship']
['looking for', 'job_search', 'professional']
['education', 'university', 'college', 'degree']
['job', 'professional', 'experience', 'work history', 'internship']
['jo